# Building Machine Learning Classifiers: Evaluate Gradient Boosting with GridSearchCV

**Grid-search:** Exhaustively search all parameter combinations in a given grid to determine the best model.

**Cross-validation:** Divide a dataset into k subsets and repeat the holdout method k times where a different subset is used as the holdout set in each iteration.

### Read in text

In [1]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

# TF-IDF
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])
X_tfidf_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)

# CountVectorizer
count_vect = CountVectorizer(analyzer=clean_text)
X_count = count_vect.fit_transform(data['body_text'])
X_count_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_count.toarray())], axis=1)

X_count_feat.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,49,4.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,62,3.2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,28,7.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,135,4.4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Exploring parameter settings using GridSearchCV

In [2]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

In [4]:
gb = GradientBoostingClassifier()
param = {
    'n_estimators': [100,150],
    'max_depth':[7,11,15],
    'learning_rate':[0.1]
}
gs=GridSearchCV(gb,param,cv=5,n_jobs=-1)
cv_fit = gs.fit(X_tfidf_feat,data['label'])
pd.DataFrame(cv_fit.cv_results_).sort_values('mean_test_score',ascending=False)[0:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
1,157.701748,0.398222,0.225413,0.014522,0.1,7,150,"{'learning_rate': 0.1, 'max_depth': 7, 'n_esti...",0.966786,0.981149,0.969452,0.964061,0.969452,0.970180,0.005837,1
5,272.627394,39.553621,0.190026,0.040014,0.1,15,150,"{'learning_rate': 0.1, 'max_depth': 15, 'n_est...",0.964991,0.975763,0.970350,0.969452,0.969452,0.970002,0.003436,2
3,250.114782,8.409172,0.220761,0.005550,0.1,11,150,"{'learning_rate': 0.1, 'max_depth': 11, 'n_est...",0.965889,0.978456,0.970350,0.965858,0.965858,0.969282,0.004904,3
0,103.786352,0.458132,0.238754,0.031077,0.1,7,100,"{'learning_rate': 0.1, 'max_depth': 7, 'n_esti...",0.964093,0.978456,0.970350,0.964960,0.967655,0.969103,0.005165,4
2,157.983490,4.721826,0.279428,0.073499,0.1,11,100,"{'learning_rate': 0.1, 'max_depth': 11, 'n_est...",0.965889,0.977558,0.967655,0.964061,0.965858,0.968204,0.004813,5


In [5]:
gb = GradientBoostingClassifier()
param = {
    'n_estimators': [100,150],
    'max_depth':[7,11,15],
    'learning_rate':[0.1]
}
gs=GridSearchCV(gb,param,cv=5,n_jobs=-1)
cv_fit = gs.fit(X_count,data['label'])
pd.DataFrame(cv_fit.cv_results_).sort_values('mean_test_score',ascending=False)[0:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
1,10.449831,0.085424,0.010388,0.000215,0.1,7,150,"{'learning_rate': 0.1, 'max_depth': 7, 'n_esti...",0.977558,0.971275,0.964061,0.964061,0.963163,0.968024,0.005595,1
5,13.952133,2.276375,0.016012,0.001851,0.1,15,150,"{'learning_rate': 0.1, 'max_depth': 15, 'n_est...",0.974865,0.969479,0.966757,0.964960,0.963163,0.967845,0.004084,2
3,13.048126,0.156294,0.014522,0.000656,0.1,11,150,"{'learning_rate': 0.1, 'max_depth': 11, 'n_est...",0.976661,0.967684,0.964061,0.967655,0.962264,0.967665,0.004961,3
4,10.879506,0.105449,0.013147,0.000411,0.1,15,100,"{'learning_rate': 0.1, 'max_depth': 15, 'n_est...",0.974865,0.968582,0.968553,0.962264,0.962264,0.967306,0.004715,4
0,7.333656,0.121778,0.008601,0.000407,0.1,7,100,"{'learning_rate': 0.1, 'max_depth': 7, 'n_esti...",0.978456,0.968582,0.960467,0.965858,0.963163,0.967305,0.006197,5
